In [40]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler

## 1. Cargamos los datos del set de entrenamiento

In [51]:
# Cargar el conjunto de datos

dataset = pd.read_csv("../data/raw/train.csv")
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [52]:
dataset.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


### 2. Porcentaje de Nulos

In [57]:
dataset.isnull().mean()

Survived    0.000000
Pclass      0.000000
Sex         0.000000
Age         0.000000
SibSp       0.000000
Parch       0.000000
Fare        0.000000
Embarked    0.002245
dtype: float64

Eliminamos la columna Cabin dado que tiene 77.1 % de faltantes

In [54]:
dataset.drop(['Cabin'], axis=1, inplace=True)

Imputamos la media en la columna Age

In [56]:
media_age = dataset['Age'].mean()
dataset['Age'] = dataset['Age'].fillna(media_age).astype(int)

Imputamos Embarked

In [58]:
dataset['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [59]:
dataset['Embarked'] = dataset['Embarked'].fillna('S')

### 3. Codificación de Variables Categóricas

In [60]:
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22,1,0,7.2500,S
1,1,1,female,38,1,0,71.2833,C
2,1,3,female,26,0,0,7.9250,S
3,1,1,female,35,1,0,53.1000,S
4,0,3,male,35,0,0,8.0500,S


In [61]:
dataset['Pclass'].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

In [62]:
dataset['SibSp'].value_counts()

SibSp
0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: count, dtype: int64

In [63]:
dataset['Sex'].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

Codificamos Variable Sex con One Hot Encoding

In [64]:
dataset['Sex'] = pd.get_dummies(dataset['Sex'], drop_first=True).astype(int)

Codificamos Variables Embarked con Frecuency Encoding

In [67]:
codificador_embarked = dataset['Embarked'].value_counts()

In [68]:
dataset['Embarked'] = dataset['Embarked'].map(codificador_embarked)

In [69]:
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22,1,0,7.2500,646
1,1,1,0,38,1,0,71.2833,168
2,1,3,0,26,0,0,7.9250,646
3,1,1,0,35,1,0,53.1000,646
4,0,3,1,35,0,0,8.0500,646


### 4. Estandarización de Variables

In [70]:
from sklearn.preprocessing import MinMaxScaler

In [72]:
mm_scaler = MinMaxScaler()

X = dataset.drop('Survived', axis=1)

scaled_feature = mm_scaler.fit_transform(X)

scaled_df = pd.DataFrame(scaled_feature, columns=X.columns)

In [73]:
scaled_df['Survived'] = dataset['Survived']
scaled_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1.0,1.0,0.2750,0.125,0.0,0.014151,1.00000,0
1,0.0,0.0,0.4750,0.125,0.0,0.139136,0.15993,1
2,1.0,0.0,0.3250,0.000,0.0,0.015469,1.00000,1
3,0.0,0.0,0.4375,0.125,0.0,0.103644,1.00000,1
4,1.0,1.0,0.4375,0.000,0.0,0.015713,1.00000,0


In [74]:
scaled_df.to_csv('../data/processed/features_for_model.csv')